In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
from tensorflow import keras
import time

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
start = time.time()

char_dict = {"char": [], "matrix": []}
char_set = dict()
src = r"dataset/handwritten_letters"
for i in range(500):
    for folder in os.listdir(src):
        char_dict["char"].append(folder)
        if folder not in char_set:
            char_set[folder] = sorted_alphanumeric(os.listdir(src + '/' + folder))
        img = cv2.imread(f"{src}/{folder}/{char_set[folder][i]}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if not img.shape == (32, 32):
            img = cv2.resize(img, (32, 32))
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        img = cv2.bitwise_not(img)
        img = img.flatten()
        char_dict["matrix"].append(img)


stop = time.time()
print(stop-start, 'seconds')

7.841053009033203 seconds


In [4]:
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
y = np.array(char_dict['char'])
y = le.fit_transform(y)

X = np.array(char_dict["matrix"])

/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [19]:
split = int(round(0.9 * len(X)))

train_X = X[:split]/255
train_y = y[:split]
test_X = X[split:]/255
test_y = y[split:]

In [20]:
train_X = train_X.astype('int32')
test_X = test_X.astype('int32')

In [21]:
values = []
for char in train_X:
    for value in char:
        if value not in values:
            values.append(value)
print('x values:', values)
values = []
for char in train_y:
    for value in char:
        if value not in values:
            values.append(value)
print('y values:', values)

x values: [0, 1]
y values: [1, 0]


In [22]:
train_X = tf.constant(train_X)
train_y = tf.constant(train_y)

test_X = tf.constant(test_X)
test_y = tf.constant(test_y)

In [23]:
train_X

<tf.Tensor: id=4114, shape=(15750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [24]:
train_y

<tf.Tensor: id=4115, shape=(15750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [25]:
test_X

<tf.Tensor: id=4116, shape=(1750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [26]:
test_y

<tf.Tensor: id=4117, shape=(1750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=1024))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(35, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [28]:
model.fit(train_X, train_y, epochs=20)

Train on 15750 samples
Epoch 1/20
15750/15750 [==============================] - 5s 314us/sample - loss: 1.3842 - accuracy: 0.6159
Epoch 2/20
15750/15750 [==============================] - 4s 285us/sample - loss: 0.5998 - accuracy: 0.8194
Epoch 3/20
15750/15750 [==============================] - 4s 284us/sample - loss: 0.4041 - accuracy: 0.8728
Epoch 4/20
15750/15750 [==============================] - 5s 286us/sample - loss: 0.2981 - accuracy: 0.9025
Epoch 5/20
15750/15750 [==============================] - 4s 284us/sample - loss: 0.2197 - accuracy: 0.9279
Epoch 6/20
15750/15750 [==============================] - 4s 286us/sample - loss: 0.1659 - accuracy: 0.9446
Epoch 7/20
15750/15750 [==============================] - 4s 285us/sample - loss: 0.1352 - accuracy: 0.9549
Epoch 8/20
15750/15750 [==============================] - 5s 286us/sample - loss: 0.1202 - accuracy: 0.9571
Epoch 9/20
15750/15750 [==============================] - 4s 285us/sample - loss: 0.1033 - accuracy: 0.9648
Epoch

In [58]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.9892698526382446% 
 Error on test data: 0.010730147361755371

Accuracy on test data: 0.8360000252723694% 
 Error on test data: 0.16399997472763062
